<a href="https://colab.research.google.com/github/payalChandile21/10xTC/blob/main/Stockpriceprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile stock.py
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pytz
import ta

##########################################################################################
## PART 1: Define Functions for Pulling, Processing, and Creating Techincial Indicators ##
##########################################################################################

# Fetch stock data based on the ticker, period, and interval
def fetch_stock_data(ticker, period, interval):
    end_date = datetime.now()
    if period == '1wk':
        start_date = end_date - timedelta(days=7)
        data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    else:
        data = yf.download(ticker, period=period, interval=interval)
    return data

# Process data to ensure it is timezone-aware and has the correct format
def process_data(data):
    if data.index.tzinfo is None:
        data.index = data.index.tz_localize('UTC')
    data.index = data.index.tz_convert('US/Eastern')
    data.reset_index(inplace=True)
    data.rename(columns={'Date': 'Datetime'}, inplace=True)
    return data

# Calculate basic metrics from the stock data
def calculate_metrics(data):
    last_close = data['Close'].iloc[-1]
    prev_close = data['Close'].iloc[0]
    change = last_close - prev_close
    pct_change = (change / prev_close) * 100
    high = data['High'].max()
    low = data['Low'].min()
    volume = data['Volume'].sum()
    return last_close, change, pct_change, high, low, volume

# Add simple moving average (SMA) and exponential moving average (EMA) indicators
def add_technical_indicators(data):
    data['SMA_20'] = ta.trend.sma_indicator(data['Close'], window=20)
    data['EMA_20'] = ta.trend.ema_indicator(data['Close'], window=20)
    return data

###############################################
## PART 2: Creating the Dashboard App layout ##
###############################################


# Set up Streamlit page layout
st.set_page_config(layout="wide")
st.title('Real Time Stock Dashboard')


# 2A: SIDEBAR PARAMETERS ############

# Sidebar for user input parameters
st.sidebar.header('Chart Parameters')
ticker = st.sidebar.text_input('Ticker', 'ADBE')
time_period = st.sidebar.selectbox('Time Period', ['1d', '1wk', '1mo', '1y', 'max'])
chart_type = st.sidebar.selectbox('Chart Type', ['Candlestick', 'Line'])
indicators = st.sidebar.multiselect('Technical Indicators', ['SMA 20', 'EMA 20'])

# Mapping of time periods to data intervals
interval_mapping = {
    '1d': '1m',
    '1wk': '30m',
    '1mo': '1d',
    '1y': '1wk',
    'max': '1wk'
}


# 2B: MAIN CONTENT AREA ############

# Update the dashboard based on user input
if st.sidebar.button('Update'):
    data = fetch_stock_data(ticker, time_period, interval_mapping[time_period])
    data = process_data(data)
    data = add_technical_indicators(data)

    last_close, change, pct_change, high, low, volume = calculate_metrics(data)

    # Display main metrics
    st.metric(label=f"{ticker} Last Price", value=f"{last_close:.2f} USD", delta=f"{change:.2f} ({pct_change:.2f}%)")

    col1, col2, col3 = st.columns(3)
    col1.metric("High", f"{high:.2f} USD")
    col2.metric("Low", f"{low:.2f} USD")
    col3.metric("Volume", f"{volume:,}")

    # Plot the stock price chart
    fig = go.Figure()
    if chart_type == 'Candlestick':
        fig.add_trace(go.Candlestick(x=data['Datetime'],
                                     open=data['Open'],
                                     high=data['High'],
                                     low=data['Low'],
                                     close=data['Close']))
    else:
        fig = px.line(data, x='Datetime', y='Close')

    # Add selected technical indicators to the chart
    for indicator in indicators:
        if indicator == 'SMA 20':
            fig.add_trace(go.Scatter(x=data['Datetime'], y=data['SMA_20'], name='SMA 20'))
        elif indicator == 'EMA 20':
            fig.add_trace(go.Scatter(x=data['Datetime'], y=data['EMA_20'], name='EMA 20'))

    # Format graph
    fig.update_layout(title=f'{ticker} {time_period.upper()} Chart',
                      xaxis_title='Time',
                      yaxis_title='Price (USD)',
                      height=600)
    st.plotly_chart(fig, use_container_width=True)

    # Display historical data and technical indicators
    st.subheader('Historical Data')
    st.dataframe(data[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']])

    st.subheader('Technical Indicators')
    st.dataframe(data[['Datetime', 'SMA_20', 'EMA_20']])


# 2C: SIDEBAR PRICES ############

# Sidebar section for real-time stock prices of selected symbols
st.sidebar.header('Real-Time Stock Prices')
stock_symbols = ['AAPL', 'GOOGL', 'AMZN', 'MSFT']
for symbol in stock_symbols:
    real_time_data = fetch_stock_data(symbol, '1d', '1m')
    if not real_time_data.empty:
        real_time_data = process_data(real_time_data)
        last_price = float(real_time_data['Close'].iloc[-1])  # Convert to float
        opening_price = float(real_time_data['Open'].iloc[0])  # Convert to float
        change = last_price - opening_price
        pct_change = (change / opening_price) * 100
        st.sidebar.metric(f"{symbol}", f"{last_price:.2f} USD", f"{change:.2f} ({pct_change:.2f}%)")

# Sidebar information section
st.sidebar.subheader('About')
st.sidebar.info('This dashboard provides stock data and technical indicators for various time periods. Use the sidebar to customize your view.')

Overwriting stock.py


In [ ]:
!pip install streamlit plotly pandas yfinance ta pytz


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.9 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=e9931b3188d7873b1c4888a337dbe3d97a919da5c98a58870b005fdb39e5ecea
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [ ]:
!pip install pyngrok

In [ ]:
!streamlit run stock.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴⠦
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.12.253:8501

⠧your url is: https://cute-planets-tease.loca.lt
  Stopping...
^C


In [ ]:
%%writefile stock.py
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pytz
import ta
import numpy as np

##########################################################################################
## PART 1: Define Functions for Pulling, Processing, and Creating Techincial Indicators ##
##########################################################################################

# Fetch stock data based on the ticker, period, and interval
def fetch_stock_data(ticker, period, interval):
    end_date = datetime.now()
    if period == '1wk':
        start_date = end_date - timedelta(days=7)
        data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    else:
        data = yf.download(ticker, period=period, interval=interval)
    return data

# Process data to ensure it is timezone-aware and has the correct format
def process_data(data):
    if data.index.tzinfo is None:
        data.index = data.index.tz_localize('UTC')
    data.index = data.index.tz_convert('US/Eastern')
    data.reset_index(inplace=True)
    data.rename(columns={'Date': 'Datetime'}, inplace=True)
    return data

# Calculate basic metrics from the stock data
def calculate_metrics(data):
    last_close = float(data['Close'].iloc[-1])
    prev_close = float(data['Close'].iloc[0])
    change = last_close - prev_close
    pct_change = (change / prev_close) * 100
    high = float(data['High'].max())
    low = float(data['Low'].min())
    volume = int(data['Volume'].sum())
    return last_close, change, pct_change, high, low, volume

# Add simple moving average (SMA) and exponential moving average (EMA) indicators
def add_technical_indicators(data):
    # Make sure data is valid for technical analysis
    if len(data) >= 20:  # Need at least window size data points
        # Convert to Series correctly
        close_series = pd.Series(data['Close'].values.squeeze())  # Ensure it's 1D

        # Create indicators
        data['SMA_20'] = ta.trend.sma_indicator(close=close_series, window=20)
        data['EMA_20'] = ta.trend.ema_indicator(close=close_series, window=20)
    else:
        # If not enough data, create empty columns
        data['SMA_20'] = float('nan')
        data['EMA_20'] = float('nan')

    return data


###############################################
## PART 2: Creating the Dashboard App layout ##
###############################################


# Set up Streamlit page layout
st.set_page_config(layout="wide")
st.title('Real Time Stock Dashboard')


# 2A: SIDEBAR PARAMETERS ############

# Sidebar for user input parameters
st.sidebar.header('Chart Parameters')
ticker = st.sidebar.text_input('Ticker', 'ADBE')
time_period = st.sidebar.selectbox('Time Period', ['1d', '1wk', '1mo', '1y', 'max'])
chart_type = st.sidebar.selectbox('Chart Type', ['Candlestick', 'Line'])
indicators = st.sidebar.multiselect('Technical Indicators', ['SMA 20', 'EMA 20'])

# Mapping of time periods to data intervals
interval_mapping = {
    '1d': '1m',
    '1wk': '30m',
    '1mo': '1d',
    '1y': '1wk',
    'max': '1wk'
}


# 2B: MAIN CONTENT AREA ############

# Update the dashboard based on user input
if st.sidebar.button('Update'):
    try:
        data = fetch_stock_data(ticker, time_period, interval_mapping[time_period])

        if not data.empty:
            data = process_data(data)
            data = add_technical_indicators(data)

            last_close, change, pct_change, high, low, volume = calculate_metrics(data)

            # Display main metrics - using explicit float conversion before formatting
            st.metric(label=f"{ticker} Last Price", value=f"{last_close:.2f} USD", delta=f"{change:.2f} ({pct_change:.2f}%)")

            col1, col2, col3 = st.columns(3)
            col1.metric("High", f"{high:.2f} USD")
            col2.metric("Low", f"{low:.2f} USD")
            col3.metric("Volume", f"{volume:,}")

            # Plot the stock price chart
            fig = go.Figure()
            if chart_type == 'Candlestick':
                fig.add_trace(go.Candlestick(x=data['Datetime'],
                                            open=data['Open'],
                                            high=data['High'],
                                            low=data['Low'],
                                            close=data['Close']))
            else:
                fig = px.line(data, x='Datetime', y='Close')

            # Add selected technical indicators to the chart
            for indicator in indicators:
                if indicator == 'SMA 20':
                    fig.add_trace(go.Scatter(x=data['Datetime'], y=data['SMA_20'], name='SMA 20'))
                elif indicator == 'EMA 20':
                    fig.add_trace(go.Scatter(x=data['Datetime'], y=data['EMA_20'], name='EMA 20'))

            # Format graph
            fig.update_layout(title=f'{ticker} {time_period.upper()} Chart',
                            xaxis_title='Time',
                            yaxis_title='Price (USD)',
                            height=600)
            st.plotly_chart(fig, use_container_width=True)

            # Display historical data and technical indicators
            st.subheader('Historical Data')
            st.dataframe(data[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']])

            st.subheader('Technical Indicators')
            st.dataframe(data[['Datetime', 'SMA_20', 'EMA_20']])
        else:
            st.error(f"No data found for {ticker}. Please check the ticker symbol and try again.")
    except Exception as e:
        st.error(f"An error occurred: {str(e)}")


# 2C: SIDEBAR PRICES ############

# Sidebar section for real-time stock prices of selected symbols
st.sidebar.header('Real-Time Stock Prices')
stock_symbols = ['AAPL', 'GOOGL', 'AMZN', 'MSFT']
for symbol in stock_symbols:
    try:
        real_time_data = fetch_stock_data(symbol, '1d', '1m')
        if not real_time_data.empty:
            real_time_data = process_data(real_time_data)
            # Explicitly convert to float before formatting
            last_price = float(real_time_data['Close'].iloc[-1])
            opening_price = float(real_time_data['Open'].iloc[0])
            change = last_price - opening_price
            pct_change = (change / opening_price) * 100
            st.sidebar.metric(f"{symbol}", f"{last_price:.2f} USD", f"{change:.2f} ({pct_change:.2f}%)")
    except Exception as e:
        st.sidebar.warning(f"Could not load data for {symbol}: {str(e)}")

# Sidebar information section
st.sidebar.subheader('About')
st.sidebar.info('This dashboard provides stock data and technical indicators for various time periods. Use the sidebar to customize your view.')

Overwriting stock.py


In [ ]:
%%writefile stock.py
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pytz
import ta
import numpy as np

##########################################################################################
## PART 1: Define Functions for Pulling, Processing, and Creating Technical Indicators ##
##########################################################################################

# Fetch stock data based on the ticker, period, and interval
def fetch_stock_data(ticker, period, interval):
    end_date = datetime.now()
    if period == '1wk':
        start_date = end_date - timedelta(days=7)
        data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    else:
        data = yf.download(ticker, period=period, interval=interval)
    return data

# Process data to ensure it is timezone-aware and formatted properly
def process_data(data):
    if data.empty:
        return data  # Return empty data safely

    data.index = pd.to_datetime(data.index)  # Ensure index is datetime
    if data.index.tzinfo is None:
        data.index = data.index.tz_localize('UTC')  # Assign UTC timezone if missing
    data.index = data.index.tz_convert('US/Eastern')  # Convert to Eastern Time
    data.reset_index(inplace=True)
    data.rename(columns={'index': 'Datetime'}, inplace=True)
    return data

# Calculate basic stock metrics
def calculate_metrics(data):
    if data.empty:
        return None, None, None, None, None, None

    last_close = float(data['Close'].iloc[-1])
    prev_close = float(data['Close'].iloc[0])
    change = last_close - prev_close
    pct_change = (change / prev_close) * 100
    high = float(data['High'].max())
    low = float(data['Low'].min())
    volume = int(data['Volume'].sum())
    return last_close, change, pct_change, high, low, volume

# Add technical indicators (SMA and EMA)
def add_technical_indicators(data):
    if data.empty or len(data) < 20:  # Ensure enough data points
        data['SMA_20'] = np.nan
        data['EMA_20'] = np.nan
        return data

    data['SMA_20'] = ta.trend.sma_indicator(close=data['Close'], window=20)
    data['EMA_20'] = ta.trend.ema_indicator(close=data['Close'], window=20)
    return data

###############################################
## PART 2: Creating the Dashboard App layout ##
###############################################

# Set up Streamlit page layout
st.set_page_config(layout="wide")
st.title('📈 Real-Time Stock Dashboard')

# 2A: SIDEBAR PARAMETERS
st.sidebar.header('⚙️ Chart Parameters')
ticker = st.sidebar.text_input('Ticker', 'AAPL')
time_period = st.sidebar.selectbox('Time Period', ['1d', '1wk', '1mo', '1y', 'max'])
chart_type = st.sidebar.selectbox('Chart Type', ['Candlestick', 'Line'])
indicators = st.sidebar.multiselect('Technical Indicators', ['SMA 20', 'EMA 20'])

# Mapping of time periods to data intervals
interval_mapping = {
    '1d': '1m',
    '1wk': '30m',
    '1mo': '1d',
    '1y': '1wk',
    'max': '1wk'
}

# 2B: MAIN CONTENT AREA
if st.sidebar.button('Update'):
    try:
        data = fetch_stock_data(ticker, time_period, interval_mapping[time_period])

        if data.empty:
            st.error(f"⚠️ No data found for {ticker}. Try another ticker or period.")
            st.stop()

        data = process_data(data)
        data = add_technical_indicators(data)

        last_close, change, pct_change, high, low, volume = calculate_metrics(data)

        # Display main metrics
        st.metric(label=f"{ticker} Last Price", value=f"{last_close:.2f} USD", delta=f"{change:.2f} ({pct_change:.2f}%)")

        col1, col2, col3 = st.columns(3)
        col1.metric("High", f"{high:.2f} USD")
        col2.metric("Low", f"{low:.2f} USD")
        col3.metric("Volume", f"{volume:,}")

        # Debugging check: Show data preview
        st.write("🔍 Data Preview:", data.head())

        # Create the stock price chart
        fig = go.Figure()

        if chart_type == 'Candlestick':
            fig.add_trace(go.Candlestick(
                x=data['Datetime'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'],
                name='Candlestick'
            ))
        else:
            fig = px.line(data, x='Datetime', y='Close', title=f'{ticker} {time_period.upper()} Closing Prices')

        # Add selected technical indicators to the chart
        for indicator in indicators:
            if indicator == 'SMA 20':
                fig.add_trace(go.Scatter(x=data['Datetime'], y=data['SMA_20'], name='SMA 20'))
            elif indicator == 'EMA 20':
                fig.add_trace(go.Scatter(x=data['Datetime'], y=data['EMA_20'], name='EMA 20'))

        # Format the graph
        fig.update_layout(
            title=f'{ticker} {time_period.upper()} Chart',
            xaxis_title='Time',
            yaxis_title='Price (USD)',
            height=600
        )
        st.plotly_chart(fig, use_container_width=True)

        # Display historical data and technical indicators
        st.subheader('📜 Historical Data')
        st.dataframe(data[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']])

        st.subheader('📊 Technical Indicators')
        st.dataframe(data[['Datetime', 'SMA_20', 'EMA_20']])

    except Exception as e:
        st.error(f"❌ An error occurred: {str(e)}")

# 2C: SIDEBAR REAL-TIME STOCK PRICES
st.sidebar.header('📡 Real-Time Stock Prices')
stock_symbols = ['AAPL', 'GOOGL', 'AMZN', 'MSFT']

for symbol in stock_symbols:
    try:
        real_time_data = fetch_stock_data(symbol, '1d', '1m')
        if real_time_data.empty:
            continue

        real_time_data = process_data(real_time_data)
        last_price = float(real_time_data['Close'].iloc[-1])
        opening_price = float(real_time_data['Open'].iloc[0])
        change = last_price - opening_price
        pct_change = (change / opening_price) * 100

        st.sidebar.metric(f"{symbol}", f"{last_price:.2f} USD", f"{change:.2f} ({pct_change:.2f}%)")

    except Exception as e:
        st.sidebar.warning(f"⚠️ Could not load data for {symbol}: {str(e)}")

# Sidebar about section
st.sidebar.subheader('ℹ️ About')
st.sidebar.info('This dashboard provides real-time stock data and technical indicators. Use the sidebar to customize your view.')


Overwriting stock.py


In [ ]:
%%writefile stock.py
import yfinance as yf
import pandas as pd
import streamlit as st
import plotly.graph_objects as go

# Function to fetch stock data
def get_stock_data(ticker, period="1mo", interval="1d"):
    stock = yf.Ticker(ticker)
    real_time_data = stock.history(period=period, interval=interval)

    if not real_time_data.empty:
        last_price = real_time_data['Close'].iloc[-1]  # Get last closing price
        opening_price = real_time_data['Open'].iloc[0]  # Get first opening price
        return real_time_data, last_price, opening_price
    else:
        return None, None, None

# Streamlit UI
st.set_page_config(page_title="Stock Market Dashboard", layout="wide")
st.title("📈 Real-Time Stock Market Dashboard")

# User input for stock ticker
ticker = st.text_input("Enter Stock Ticker Symbol (e.g., AAPL, TSLA, GOOG):", "AAPL")

if st.button("Get Stock Data"):
    data, last_price, opening_price = get_stock_data(ticker)

    if data is not None:
        st.markdown(f"### **Stock Data for {ticker}**")
        col1, col2 = st.columns(2)

        with col1:
            st.metric(label="📌 Last Price", value=f"${last_price:.2f}")
        with col2:
            st.metric(label="📌 Opening Price", value=f"${opening_price:.2f}")

        # Line Chart
        st.subheader("📊 Stock Price Trend")
        fig_line = go.Figure()
        fig_line.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Close Price', line=dict(color='blue')))
        fig_line.update_layout(title=f"Stock Price Trend for {ticker}", xaxis_title="Date", yaxis_title="Price")
        st.plotly_chart(fig_line, use_container_width=True)

        # Candlestick Chart
        st.subheader("🕯️ Candlestick Chart")
        fig_candle = go.Figure(data=[go.Candlestick(
            x=data.index,
            open=data['Open'],
            high=data['High'],
            low=data['Low'],
            close=data['Close'],
            name="Candlestick"
        )])
        fig_candle.update_layout(title=f"Candlestick Chart for {ticker}", xaxis_title="Date", yaxis_title="Price")
        st.plotly_chart(fig_candle, use_container_width=True)

        # Show Raw Data
        with st.expander("📜 View Raw Data"):
            st.dataframe(data)

    else:
        st.error("Stock data could not be retrieved. Please check the ticker symbol.")



Overwriting stock.py


In [ ]:
%%writefile stock.py
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pytz
import ta

# Fetch stock data based on the ticker, period, and interval
def fetch_stock_data(ticker, period, interval):
    end_date = datetime.now()
    if period == '1wk':
        start_date = end_date - timedelta(days=7)
        data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    else:
        data = yf.download(ticker, period=period, interval=interval)
    return data

# Process data to ensure it is timezone-aware and has the correct format
def process_data(data):
    if data.index.tzinfo is None:
        data.index = data.index.tz_localize('UTC')
    data.index = data.index.tz_convert('US/Eastern')
    data.reset_index(inplace=True)
    data.rename(columns={'Date': 'Datetime'}, inplace=True)
    return data

# Calculate basic metrics from the stock data
def calculate_metrics(data):
    last_close = float(data['Close'].iloc[-1])  # Ensure float conversion
    prev_close = float(data['Close'].iloc[0])
    change = last_close - prev_close
    pct_change = (change / prev_close) * 100
    high = float(data['High'].max())
    low = float(data['Low'].min())
    volume = float(data['Volume'].sum())
    return last_close, change, pct_change, high, low, volume

# Add technical indicators (SMA, EMA, RSI, MACD)
def add_technical_indicators(data):
    data['SMA_20'] = ta.trend.sma_indicator(data['Close'], window=20)
    data['EMA_20'] = ta.trend.ema_indicator(data['Close'], window=20)
    data['RSI'] = ta.momentum.rsi(data['Close'], window=14)  # Added RSI calculation
    data['MACD'] = ta.trend.macd(data['Close'])  # Added MACD calculation
    return data

st.set_page_config(layout="wide")
st.title('Real Time Stock Dashboard')

# Sidebar parameters
st.sidebar.header('Chart Parameters')
ticker = st.sidebar.text_input('Ticker', 'ADBE')
time_period = st.sidebar.selectbox('Time Period', ['1d', '1wk', '1mo', '1y', 'max'])
chart_type = st.sidebar.selectbox('Chart Type', ['Candlestick', 'Line'])
indicators = st.sidebar.multiselect('Technical Indicators', ['SMA 20', 'EMA 20', 'RSI', 'MACD'])

interval_mapping = {
    '1d': '1m',
    '1wk': '30m',
    '1mo': '1d',
    '1y': '1wk',
    'max': '1wk'
}

if st.sidebar.button('Update'):
    data = fetch_stock_data(ticker, time_period, interval_mapping[time_period])
    data = process_data(data)
    data = add_technical_indicators(data)
    last_close, change, pct_change, high, low, volume = calculate_metrics(data)

    st.metric(label=f"{ticker} Last Price", value=f"{last_close:.2f} USD", delta=f"{change:.2f} ({pct_change:.2f}%)")

    col1, col2, col3 = st.columns(3)
    col1.metric("High", f"{high:.2f} USD")
    col2.metric("Low", f"{low:.2f} USD")
    col3.metric("Volume", f"{volume:,}")

    # Plot price chart
    fig = go.Figure()
    if chart_type == 'Candlestick':
        fig.add_trace(go.Candlestick(x=data['Datetime'],
                                     open=data['Open'],
                                     high=data['High'],
                                     low=data['Low'],
                                     close=data['Close']))
    else:
        fig = px.line(data, x='Datetime', y='Close')

    for indicator in indicators:
        if indicator == 'SMA 20':
            fig.add_trace(go.Scatter(x=data['Datetime'], y=data['SMA_20'], name='SMA 20'))
        elif indicator == 'EMA 20':
            fig.add_trace(go.Scatter(x=data['Datetime'], y=data['EMA_20'], name='EMA 20'))

    fig.update_layout(title=f'{ticker} {time_period.upper()} Chart', xaxis_title='Time', yaxis_title='Price (USD)', height=600)
    st.plotly_chart(fig, use_container_width=True)

    # Additional visualizations
    if 'RSI' in indicators:
        st.subheader('RSI Indicator')
        fig_rsi = px.line(data, x='Datetime', y='RSI', title='Relative Strength Index (RSI)')
        st.plotly_chart(fig_rsi, use_container_width=True)

    if 'MACD' in indicators:
        st.subheader('MACD Indicator')
        fig_macd = px.line(data, x='Datetime', y='MACD', title='MACD Indicator')
        st.plotly_chart(fig_macd, use_container_width=True)

    st.subheader('Historical Data')
    st.dataframe(data[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']])

    st.subheader('Technical Indicators')
    st.dataframe(data[['Datetime', 'SMA_20', 'EMA_20', 'RSI', 'MACD']])

# Real-time stock prices
st.sidebar.header('Real-Time Stock Prices')
stock_symbols = ['AAPL', 'GOOGL', 'AMZN', 'MSFT']
for symbol in stock_symbols:
    real_time_data = fetch_stock_data(symbol, '1d', '1m')
    if not real_time_data.empty:
        real_time_data = process_data(real_time_data)
        last_price = float(real_time_data['Close'].iloc[-1])
        opening_price = float(real_time_data['Open'].iloc[0])
        change = last_price - opening_price
        pct_change = (change / opening_price) * 100
        st.sidebar.metric(f"{symbol}", f"{last_price:.2f} USD", f"{change:.2f} ({pct_change:.2f}%)")

st.sidebar.subheader('About')
st.sidebar.info('This dashboard provides stock data and technical indicators for various time periods. Use the sidebar to customize your view.')


Overwriting stock.py


In [ ]:
!streamlit run stock.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.12.253:8501

⠴your url is: https://olive-maps-drive.loca.lt
YF.download() has changed argument auto_adjust default to True
[*********************100%***********************]  1 of 1 completed
/content/stock.py:123: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  last_price = float(real_time_data['Close'].iloc[-1])
/content/stock.py:124: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  opening_price = float(real_time_data['Open'].iloc[0])
[*********************100%***********************]  1 of 1 completed
/content/stock.py:123: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the fu

In [ ]:
%%writefile tech.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands

# Load Data (User should upload CSV)
st.title("Stock Market Technical Analysis")
uploaded_file = st.file_uploader("Upload a CSV file with 'Close' prices", type=["csv"])

if uploaded_file is not None:
    df = pd.read_csv(uploaded_file, index_col=0, parse_dates=True)
    st.write("Data Preview:", df.head())

    # Trend Analysis
    def trend(x):
        if -0.5 < x <= 0.5: return 'Slight or No change'
        elif 0.5 < x <= 1: return 'Slight Positive'
        elif -1 < x <= -0.5: return 'Slight Negative'
        elif 1 < x <= 3: return 'Positive'
        elif -3 < x <= -1: return 'Negative'
        elif 3 < x <= 7: return 'Among top gainers'
        elif -7 < x <= -3: return 'Among top losers'
        elif x > 7: return 'Bull run'
        else: return 'Bear drop'

    df['Trend'] = df['Close'].pct_change().apply(lambda x: trend(x))
    st.subheader("Trend Analysis")
    fig, ax = plt.subplots()
    df['Trend'].value_counts().plot(kind='pie', autopct='%1.1f%%', ax=ax)
    ax.set_ylabel('')
    st.pyplot(fig)

    # Moving Averages
    st.subheader("Simple & Exponential Moving Averages")
    df['50_SMA'] = df['Close'].rolling(window=50, min_periods=1).mean()
    df['200_SMA'] = df['Close'].rolling(window=200, min_periods=1).mean()
    df['50_EMA'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['200_EMA'] = df['Close'].ewm(span=200, adjust=False).mean()

    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(df.index, df['Close'], label='Close Price', color='black')
    ax.plot(df.index, df['50_SMA'], label='50-day SMA', color='blue')
    ax.plot(df.index, df['200_SMA'], label='200-day SMA', color='green')
    ax.plot(df.index, df['50_EMA'], label='50-day EMA', linestyle='dashed', color='blue')
    ax.plot(df.index, df['200_EMA'], label='200-day EMA', linestyle='dashed', color='green')
    ax.legend()
    st.pyplot(fig)

    # RSI
    st.subheader("Relative Strength Index (RSI)")
    df['RSI'] = RSIIndicator(df['Close'], window=14).rsi()
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(df.index, df['RSI'], label='RSI', color='blue')
    ax.axhline(30, linestyle='--', color='red')
    ax.axhline(70, linestyle='--', color='green')
    ax.legend()
    st.pyplot(fig)

    # Bollinger Bands
    st.subheader("Bollinger Bands")
    indicator_bb = BollingerBands(df['Close'], window=20, window_dev=2)
    df['BB_Middle'] = indicator_bb.bollinger_mavg()
    df['BB_Upper'] = indicator_bb.bollinger_hband()
    df['BB_Lower'] = indicator_bb.bollinger_lband()

    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(df.index, df['Close'], label='Close Price', color='black')
    ax.plot(df.index, df['BB_Middle'], label='Middle Band', color='blue')
    ax.plot(df.index, df['BB_Upper'], label='Upper Band', color='green')
    ax.plot(df.index, df['BB_Lower'], label='Lower Band', color='red')
    ax.fill_between(df.index, df['BB_Lower'], df['BB_Upper'], color='gray', alpha=0.3)
    ax.legend()
    st.pyplot(fig)


Overwriting tech.py


In [ ]:
!streamlit run techy.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.12.253:8501

⠼⠴⠦your url is: https://lazy-facts-search.loca.lt
  Stopping...
^C


In [ ]:
%%writefile techy.py
import streamlit as st
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands

# Function to fetch stock data
def get_stock_data(ticker, period='1y'):
    stock = yf.Ticker(ticker)
    df = stock.history(period=period)
    return df

# Function to calculate trends
def trend(x):
    if x > -0.5 and x <= 0.5:
        return 'Slight or No change'
    elif x > 0.5 and x <= 1:
        return 'Slight Positive'
    elif x > -1 and x <= -0.5:
        return 'Slight Negative'
    elif x > 1 and x <= 3:
        return 'Positive'
    elif x > -3 and x <= -1:
        return 'Negative'
    elif x > 3 and x <= 7:
        return 'Among top gainers'
    elif x > -7 and x <= -3:
        return 'Among top losers'
    elif x > 7:
        return 'Bull run'
    elif x <= -7:
        return 'Bear drop'

# Streamlit App
st.title("Stock Market Technical Analysis")
ticker = st.text_input("Enter Stock Ticker (e.g., AAPL, TSLA, MSFT)", "AAPL")
period = st.selectbox("Select Time Period", ['1mo', '3mo', '6mo', '1y', '2y', '5y'], index=3)

if st.button("Analyze"):
    df = get_stock_data(ticker, period)
    df['Day_Perc_Change'] = df['Close'].pct_change() * 100
    df.dropna(inplace=True)
    df['Trend'] = df['Day_Perc_Change'].apply(lambda x: trend(x))

    # Plot Trends
    st.subheader("Market Trend Analysis")
    fig, ax = plt.subplots()
    df['Trend'].value_counts().plot.pie(autopct='%1.1f%%', ax=ax)
    st.pyplot(fig)

    # Moving Averages
    st.subheader("Moving Averages")
    df['50_SMA'] = df['Close'].rolling(window=50, min_periods=1).mean()
    df['200_SMA'] = df['Close'].rolling(window=200, min_periods=1).mean()
    fig, ax = plt.subplots()
    df['Close'].plot(ax=ax, color='k', label='Close Price')
    df['50_SMA'].plot(ax=ax, color='b', label='50-day SMA')
    df['200_SMA'].plot(ax=ax, color='g', label='200-day SMA')
    ax.legend()
    st.pyplot(fig)

    # RSI Indicator
    st.subheader("Relative Strength Index (RSI)")
    rsi = RSIIndicator(df['Close'], window=14)
    df['RSI'] = rsi.rsi()
    fig, ax = plt.subplots()
    ax.plot(df.index, df['RSI'], label='RSI', color='blue')
    ax.axhline(70, linestyle='--', color='red')
    ax.axhline(30, linestyle='--', color='green')
    ax.legend()
    st.pyplot(fig)

    # Bollinger Bands
    st.subheader("Bollinger Bands")
    bb = BollingerBands(df['Close'], window=20, window_dev=2)
    df['BB_Middle'] = bb.bollinger_mavg()
    df['BB_Upper'] = bb.bollinger_hband()
    df['BB_Lower'] = bb.bollinger_lband()
    fig, ax = plt.subplots()
    ax.plot(df.index, df['Close'], label='Close Price', color='k')
    ax.plot(df.index, df['BB_Upper'], label='Upper Band', color='r')
    ax.plot(df.index, df['BB_Lower'], label='Lower Band', color='g')
    ax.fill_between(df.index, df['BB_Lower'], df['BB_Upper'], alpha=0.1)
    ax.legend()
    st.pyplot(fig)

    st.write("Data Preview:")
    st.dataframe(df.tail())


Writing techy.py


In [ ]:
%%writefile techad.py
import streamlit as st
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands

def fetch_data(ticker, start, end):
    df = yf.download(ticker, start=start, end=end)
    return df

def calculate_trend(x):
    if x > -0.5 and x <= 0.5:
        return 'Hold'
    elif x > 0.5 and x <= 1:
        return 'Slight Positive'
    elif x > -1 and x <= -0.5:
        return 'Slight Negative'
    elif x > 1 and x <= 3:
        return 'Positive'
    elif x > -3 and x <= -1:
        return 'Negative'
    elif x > 3:
        return 'Buy'
    elif x < -3:
        return 'Sell'

def add_indicators(df):
    df['50_SMA'] = df['Close'].rolling(window=50).mean()
    df['200_SMA'] = df['Close'].rolling(window=200).mean()
    df['Signal'] = np.where(df['50_SMA'] > df['200_SMA'], 'Buy', 'Sell')
    df['RSI'] = RSIIndicator(df['Close'].squeeze(), window=14).rsi()
    bb = BollingerBands(close=df['Close'].squeeze(), window=20, window_dev=2)
    df['BB_Upper'] = bb.bollinger_hband().squeeze()  # Ensure it's 1D
    df['BB_Lower'] = bb.bollinger_lband().squeeze()  # Ensure it's 1D


    return df

def plot_chart(df, ticker):
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df['Close'], label='Close Price', color='black')
    plt.plot(df.index, df['50_SMA'], label='50-day SMA', color='blue')
    plt.plot(df.index, df['200_SMA'], label='200-day SMA', color='green')
    plt.scatter(df.index[df['Signal'] == 'Buy'], df['50_SMA'][df['Signal'] == 'Buy'], marker='^', color='g', label='Buy Signal')
    plt.scatter(df.index[df['Signal'] == 'Sell'], df['200_SMA'][df['Signal'] == 'Sell'], marker='v', color='r', label='Sell Signal')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(f'Technical Analysis for {ticker}')
    plt.legend()
    plt.grid()
    st.pyplot(plt)

def main():
    st.title("Stock Market Technical Analysis")
    ticker = st.text_input("Enter Stock Ticker (e.g. AAPL, TSLA, MSFT)", 'AAPL')
    start_date = st.date_input("Start Date", pd.to_datetime('2023-01-01'))
    end_date = st.date_input("End Date", pd.to_datetime('2024-01-01'))
    if st.button("Analyze"):
        df = fetch_data(ticker, start_date, end_date)
        df = add_indicators(df)
        st.write("### Stock Data with Indicators")
        st.dataframe(df.tail())
        plot_chart(df, ticker)

if __name__ == "__main__":
    main()


Overwriting techad.py


In [ ]:
!streamlit run techy.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.12.253:8501

⠼⠴your url is: https://cool-mails-admire.loca.lt
  Stopping...
^C


In [ ]:
%%writefile mcad.py
import streamlit as st
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import MACD

# Function to fetch stock data
def get_stock_data(ticker, period='1y'):
    stock = yf.Ticker(ticker)
    df = stock.history(period=period)
    return df

# Function to calculate buy and sell signals using MACD
def macd_signals(df):
    macd = MACD(df['Close'])
    df['MACD'] = macd.macd()
    df['Signal_Line'] = macd.macd_signal()
    df['Buy_Signal'] = (df['MACD'] > df['Signal_Line']) & (df['MACD'].shift(1) <= df['Signal_Line'].shift(1))
    df['Sell_Signal'] = (df['MACD'] < df['Signal_Line']) & (df['MACD'].shift(1) >= df['Signal_Line'].shift(1))
    return df

# Streamlit App
st.title("Stock Market Technical Analysis")
ticker = st.text_input("Enter Stock Ticker (e.g., AAPL, TSLA, MSFT)", "AAPL")
period = st.selectbox("Select Time Period", ['1mo', '3mo', '6mo', '1y', '2y', '5y'], index=3)

if st.button("Analyze"):
    df = get_stock_data(ticker, period)
    df = macd_signals(df)

    # Plot Stock Price with Buy/Sell Signals
    st.subheader("Stock Price with Buy/Sell Signals")
    fig, ax = plt.subplots()
    ax.plot(df.index, df['Close'], label='Close Price', color='black')
    ax.scatter(df.index[df['Buy_Signal']], df['Close'][df['Buy_Signal']], marker='^', color='green', label='Buy Signal', alpha=1)
    ax.scatter(df.index[df['Sell_Signal']], df['Close'][df['Sell_Signal']], marker='v', color='red', label='Sell Signal', alpha=1)
    ax.legend()
    st.pyplot(fig)

    # Show Data Preview
    st.write("Data Preview:")
    st.dataframe(df.tail())


Writing mcad.py


In [ ]:
!streamlit run mcad.py & npx localtunnel --port 8501



⠙⠹
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.12.253:8501

⠸⠼⠴⠦⠧⠇⠏⠋⠙your url is: https://tender-windows-win.loca.lt
  Stopping...
^C


In [ ]:
%%writefile technicalanalysis.py
import streamlit as st
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import MACD

# Function to fetch stock data
def get_stock_data(ticker, period='1y'):
    stock = yf.Ticker(ticker)
    df = stock.history(period=period)
    return df

# Function to calculate trends
def trend(x):
    if x > -0.5 and x <= 0.5:
        return 'Slight or No change'
    elif x > 0.5 and x <= 1:
        return 'Slight Positive'
    elif x > -1 and x <= -0.5:
        return 'Slight Negative'
    elif x > 1 and x <= 3:
        return 'Positive'
    elif x > -3 and x <= -1:
        return 'Negative'
    elif x > 3 and x <= 7:
        return 'Among top gainers'
    elif x > -7 and x <= -3:
        return 'Among top losers'
    elif x > 7:
        return 'Bull run'
    elif x <= -7:
        return 'Bear drop'

# Function to calculate RSI buy and sell signals
def rsi_signals(df):
    rsi = RSIIndicator(df['Close'], window=14)
    df['RSI'] = rsi.rsi()
    df['Buy_Signal_RSI'] = df['RSI'] < 40
    df['Sell_Signal_RSI'] = df['RSI'] > 70
    return df

# Function to calculate Bollinger Bands buy and sell signals
def bollinger_signals(df):
    bb = BollingerBands(df['Close'], window=20, window_dev=2)
    df['BB_Upper'] = bb.bollinger_hband()
    df['BB_Lower'] = bb.bollinger_lband()
    df['Buy_Signal_BB'] = df['Close'] < df['BB_Lower']
    df['Sell_Signal_BB'] = df['Close'] > df['BB_Upper']
    return df

# Function to calculate MACD buy and sell signals
def macd_signals(df):
    macd = MACD(df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    df['Buy_Signal_MACD'] = df['MACD'] > df['MACD_Signal']
    df['Sell_Signal_MACD'] = df['MACD'] < df['MACD_Signal']
    return df

# Streamlit App
st.title("Stock Market Technical Analysis")
ticker = st.text_input("Enter Stock Ticker (e.g., AAPL, TSLA, MSFT)", "AAPL")
period = st.selectbox("Select Time Period", ['1mo', '3mo', '6mo', '1y', '2y', '5y'], index=3)

if st.button("Analyze"):
    df = get_stock_data(ticker, period)
    df['Day_Perc_Change'] = df['Close'].pct_change() * 100
    df.dropna(inplace=True)
    df['Trend'] = df['Day_Perc_Change'].apply(lambda x: trend(x))
    df = rsi_signals(df)
    df = bollinger_signals(df)
    df = macd_signals(df)

    # Plot Market Trends
    st.subheader("Market Trend Analysis")
    fig, ax = plt.subplots()
    df['Trend'].value_counts().plot.pie(autopct='%1.1f%%', ax=ax)
    st.pyplot(fig)

    # Plot RSI with Buy/Sell Signals
    st.subheader("RSI Indicator with Buy/Sell Signals")
    fig, ax = plt.subplots()
    ax.plot(df.index, df['RSI'], label='RSI', color='blue')
    ax.axhline(70, linestyle='--', color='red', label='Overbought')
    ax.axhline(30, linestyle='--', color='green', label='Oversold')
    ax.scatter(df.index[df['Buy_Signal_RSI']], df['RSI'][df['Buy_Signal_RSI']], marker='^', color='green', label='Buy Signal', alpha=1)
    ax.scatter(df.index[df['Sell_Signal_RSI']], df['RSI'][df['Sell_Signal_RSI']], marker='v', color='red', label='Sell Signal', alpha=1)
    ax.legend()
    st.pyplot(fig)

    # Plot Bollinger Bands with Buy/Sell Signals
    st.subheader("Bollinger Bands with Buy/Sell Signals")
    fig, ax = plt.subplots()
    ax.plot(df.index, df['Close'], label='Close Price', color='black')
    ax.plot(df.index, df['BB_Upper'], label='Upper Band', color='red')
    ax.plot(df.index, df['BB_Lower'], label='Lower Band', color='green')
    ax.fill_between(df.index, df['BB_Lower'], df['BB_Upper'], alpha=0.1)
    ax.scatter(df.index[df['Buy_Signal_BB']], df['Close'][df['Buy_Signal_BB']], marker='^', color='green', label='Buy Signal', alpha=1)
    ax.scatter(df.index[df['Sell_Signal_BB']], df['Close'][df['Sell_Signal_BB']], marker='v', color='red', label='Sell Signal', alpha=1)
    ax.legend()
    st.pyplot(fig)

    # Plot MACD with Buy/Sell Signals
    st.subheader("MACD Indicator with Buy/Sell Signals")
    fig, ax = plt.subplots()
    ax.plot(df.index, df['MACD'], label='MACD', color='blue')
    ax.plot(df.index, df['MACD_Signal'], label='Signal Line', color='red')
    ax.scatter(df.index[df['Buy_Signal_MACD']], df['MACD'][df['Buy_Signal_MACD']], marker='^', color='green', label='Buy Signal', alpha=1)
    ax.scatter(df.index[df['Sell_Signal_MACD']], df['MACD'][df['Sell_Signal_MACD']], marker='v', color='red', label='Sell Signal', alpha=1)
    ax.legend()
    st.pyplot(fig)

    # Moving Averages
    st.subheader("Moving Averages")
    df['50_SMA'] = df['Close'].rolling(window=50, min_periods=1).mean()
    df['200_SMA'] = df['Close'].rolling(window=200, min_periods=1).mean()
    fig, ax = plt.subplots()
    df['Close'].plot(ax=ax, color='k', label='Close Price')
    df['50_SMA'].plot(ax=ax, color='b', label='50-day SMA')
    df['200_SMA'].plot(ax=ax, color='g', label='200-day SMA')
    ax.legend()
    st.pyplot(fig)

    st.write("Data Preview:")
    st.dataframe(df.tail())


Overwriting technicalanalysis.py


In [ ]:
!streamlit run stockta.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.12.253:8501

⠼⠴⠦your url is: https://evil-brooms-retire.loca.lt
/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:29585 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:517:28)
    at emitErrorNT (node:internal/streams/destroy:151:8)
    at emitErrorCloseNT (node:internal/streams/destroy:116:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v18.20.5
⠙  Stopping...


In [ ]:
%%writefile bmcad.py
import streamlit as st
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import MACD

# Function to fetch stock data
def get_stock_data(ticker, period='1y'):
    stock = yf.Ticker(ticker)
    df = stock.history(period=period)
    return df

# Function to calculate buy and sell signals using MACD
def macd_signals(df):
    macd = MACD(df['Close'])
    df['MACD'] = macd.macd()
    df['Signal_Line'] = macd.macd_signal()
    df['Buy_Signal_MACD'] = (df['MACD'] > df['Signal_Line']) & (df['MACD'].shift(1) <= df['Signal_Line'].shift(1))
    df['Sell_Signal_MACD'] = (df['MACD'] < df['Signal_Line']) & (df['MACD'].shift(1) >= df['Signal_Line'].shift(1))
    return df

# Function to calculate buy and sell signals using Bollinger Bands
def bollinger_signals(df):
    indicator_bb = BollingerBands(df['Close'])
    df['BB_High'] = indicator_bb.bollinger_hband()
    df['BB_Low'] = indicator_bb.bollinger_lband()
    df['Buy_Signal_BB'] = df['Close'] < df['BB_Low']
    df['Sell_Signal_BB'] = df['Close'] > df['BB_High']
    return df

# Streamlit App
st.title("Stock Market Technical Analysis")
ticker = st.text_input("Enter Stock Ticker (e.g., AAPL, TSLA, MSFT)", "AAPL")
period = st.selectbox("Select Time Period", ['1mo', '3mo', '6mo', '1y', '2y', '5y'], index=3)

if st.button("Analyze"):
    df = get_stock_data(ticker, period)
    df = macd_signals(df)
    df = bollinger_signals(df)

    # Plot Stock Price with MACD Buy/Sell Signals
    st.subheader("Stock Price with MACD Buy/Sell Signals")
    fig, ax = plt.subplots()
    ax.plot(df.index, df['Close'], label='Close Price', color='black')
    ax.scatter(df.index[df['Buy_Signal_MACD']], df['Close'][df['Buy_Signal_MACD']], marker='^', color='green', label='Buy Signal (MACD)', alpha=1)
    ax.scatter(df.index[df['Sell_Signal_MACD']], df['Close'][df['Sell_Signal_MACD']], marker='v', color='red', label='Sell Signal (MACD)', alpha=1)
    ax.legend()
    st.pyplot(fig)

    # Plot Stock Price with Bollinger Bands and Buy/Sell Signals
    st.subheader("Stock Price with Bollinger Bands Buy/Sell Signals")
    fig, ax = plt.subplots()
    ax.plot(df.index, df['Close'], label='Close Price', color='black')
    ax.plot(df.index, df['BB_High'], label='Upper Bollinger Band', linestyle='dashed', color='blue')
    ax.plot(df.index, df['BB_Low'], label='Lower Bollinger Band', linestyle='dashed', color='blue')
    ax.scatter(df.index[df['Buy_Signal_BB']], df['Close'][df['Buy_Signal_BB']], marker='^', color='green', label='Buy Signal (BB)', alpha=1)
    ax.scatter(df.index[df['Sell_Signal_BB']], df['Close'][df['Sell_Signal_BB']], marker='v', color='red', label='Sell Signal (BB)', alpha=1)
    ax.legend()
    st.pyplot(fig)

    # Show Data Preview
    st.write("Data Preview:")
    st.dataframe(df.tail())


Overwriting bmcad.py


In [ ]:
!streamlit run bmcad.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.12.253:8501

⠴your url is: https://itchy-bags-decide.loca.lt
  Stopping...
^C


In [ ]:
%%writefile stcokfinal.py
import streamlit as st
from datetime import date, datetime, timedelta
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.plot import plot_plotly
from plotly import graph_objs as go
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import MACD

# Configure page
st.set_page_config(
    page_title="Stock Analysis & Forecast App",
    page_icon="📈",
    layout="wide"
)

# App title and description
st.title('📈 Stock Analysis & Forecast App')
st.markdown("""
This app provides comprehensive stock analysis using technical indicators and forecasts future prices using Facebook's Prophet model.
""")

# Define available stock options
stocks = {
    "Google": "GOOGL",
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Tesla": "TSLA",
    "Tata Motors": "TATAMOTORS.NS"
}

def fetch_data_from_yfinance(ticker, period='5y'):
    """Fetch stock data using yfinance"""
    try:
        # Get data for the specified period
        end_date = datetime.now()

        if period == '5y' or period == '1825d':
            start_date = end_date - timedelta(days=1825)
        else:
            # For technical analysis periods
            df = yf.download(ticker, period=period)
            return df

        # Download data for forecast
        df = yf.download(ticker, start=start_date, end=end_date)

        if not df.empty and len(df) > 30:
            return df
        return None
    except Exception as e:
        st.error(f"Error fetching data from Yahoo Finance: {str(e)}")
        return None

def get_sample_data(ticker):
    """Generate sample stock data for demonstration"""
    seed_value = sum(ord(c) for c in ticker)
    np.random.seed(seed_value)

    end_date = datetime.now()
    start_date = end_date - timedelta(days=1825)
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')

    # Set initial price based on ticker
    start_price = {
        'AAPL': 150,
        'GOOGL': 2800,
        'MSFT': 300,
        'TSLA': 800
    }.get(ticker, 100)

    # Generate price data
    daily_returns = np.random.normal(0.0005, 0.02, len(date_range))
    price_series = start_price * (1 + daily_returns).cumprod()

    # Create DataFrame
    df = pd.DataFrame(index=date_range)
    df['Close'] = price_series
    df['Open'] = df['Close'].shift(1) * (1 + np.random.normal(0, 0.005, len(df)))
    df['High'] = df[['Open', 'Close']].max(axis=1) * (1 + abs(np.random.normal(0, 0.003, len(df))))
    df['Low'] = df[['Open', 'Close']].min(axis=1) * (1 - abs(np.random.normal(0, 0.003, len(df))))
    df['Adj Close'] = df['Close']
    df['Volume'] = np.random.randint(100000, 10000000, len(df))

    return df.fillna(method='bfill')

def prepare_dataframe(df):
    """Prepare dataframe for Prophet model"""
    df = df.reset_index()
    result_df = pd.DataFrame()
    result_df['ds'] = df['Date']
    result_df['y'] = df['Close']
    return result_df

@st.cache_data(show_spinner=False)
def fetch_stock_data(stock_ticker, period='5y'):
    """Fetch stock data with fallback to sample data"""
    with st.spinner(f"Fetching data for {stock_ticker}..."):
        df = fetch_data_from_yfinance(stock_ticker, period)
        if df is not None and not df.empty and len(df) > 30:
            st.success(f"Successfully loaded data for {stock_ticker}")
            if period == '5y' or period == '1825d':
                return prepare_dataframe(df), df
            return df

        st.warning(f"Using sample data for {stock_ticker}")
        sample_data = get_sample_data(stock_ticker)
        if period == '5y' or period == '1825d':
            return prepare_dataframe(sample_data), sample_data
        return sample_data

# Technical Analysis Functions
def trend(x):
    if x > -0.5 and x <= 0.5:
        return 'Slight or No change'
    elif x > 0.5 and x <= 1:
        return 'Slight Positive'
    elif x > -1 and x <= -0.5:
        return 'Slight Negative'
    elif x > 1 and x <= 3:
        return 'Positive'
    elif x > -3 and x <= -1:
        return 'Negative'
    elif x > 3 and x <= 7:
        return 'Among top gainers'
    elif x > -7 and x <= -3:
        return 'Among top losers'
    elif x > 7:
        return 'Bull run'
    elif x <= -7:
        return 'Bear drop'

def rsi_signals(df):
    rsi = RSIIndicator(df['Close'], window=14)
    df['RSI'] = rsi.rsi()
    df['Buy_Signal_RSI'] = df['RSI'] < 40
    df['Sell_Signal_RSI'] = df['RSI'] > 70
    return df

def bollinger_signals(df):
    bb = BollingerBands(df['Close'], window=20, window_dev=2)
    df['BB_Upper'] = bb.bollinger_hband()
    df['BB_Lower'] = bb.bollinger_lband()
    df['Buy_Signal_BB'] = df['Close'] < df['BB_Lower']
    df['Sell_Signal_BB'] = df['Close'] > df['BB_Upper']
    return df

def macd_signals(df):
    macd = MACD(df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    df['Buy_Signal_MACD'] = df['MACD'] > df['MACD_Signal']
    df['Sell_Signal_MACD'] = df['MACD'] < df['MACD_Signal']
    return df

def perform_technical_analysis(df):
    """Calculate all technical indicators and signals"""
    df['Day_Perc_Change'] = df['Close'].pct_change() * 100
    df.dropna(inplace=True)
    df['Trend'] = df['Day_Perc_Change'].apply(lambda x: trend(x))
    df = rsi_signals(df)
    df = bollinger_signals(df)
    df = macd_signals(df)

    # Add moving averages
    df['50_SMA'] = df['Close'].rolling(window=50, min_periods=1).mean()
    df['200_SMA'] = df['Close'].rolling(window=200, min_periods=1).mean()
    return df

def display_technical_analysis(df, ticker):
    """Display technical analysis charts and data"""
    st.header(f"Technical Analysis for {ticker}")

    # Market Trend Analysis
    st.subheader("Market Trend Analysis")
    col1, col2 = st.columns(2)
    with col1:
        fig, ax = plt.subplots(figsize=(8, 6))
        df['Trend'].value_counts().plot.pie(autopct='%1.1f%%', ax=ax, colors=plt.cm.Paired.colors)
        plt.title('Distribution of Price Movements')
        st.pyplot(fig)

    with col2:
        st.markdown("### Trend Interpretation")
        st.write("""
        This pie chart shows the distribution of daily price movements categorized by trend strength.
        - **Bull run/Bear drop**: Extreme price movements (>7%)
        - **Top gainers/losers**: Strong movements (3-7%)
        - **Positive/Negative**: Moderate movements (1-3%)
        - **Slight changes**: Minor movements (0.5-1%)
        - **No change**: Minimal price movement (<0.5%)
        """)

    # RSI with Buy/Sell Signals
    st.subheader("RSI Indicator with Buy/Sell Signals")
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df.index, df['RSI'], label='RSI', color='blue')
    ax.axhline(70, linestyle='--', color='red', label='Overbought (70)')
    ax.axhline(30, linestyle='--', color='green', label='Oversold (30)')
    ax.scatter(df.index[df['Buy_Signal_RSI']], df['RSI'][df['Buy_Signal_RSI']],
              marker='^', color='green', label='Buy Signal', alpha=1)
    ax.scatter(df.index[df['Sell_Signal_RSI']], df['RSI'][df['Sell_Signal_RSI']],
              marker='v', color='red', label='Sell Signal', alpha=1)
    ax.set_title('Relative Strength Index (RSI)')
    ax.set_ylabel('RSI Value')
    ax.legend()
    ax.grid(True, alpha=0.3)
    st.pyplot(fig)

    # Bollinger Bands with Buy/Sell Signals
    st.subheader("Bollinger Bands with Buy/Sell Signals")
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df.index, df['Close'], label='Close Price', color='black')
    ax.plot(df.index, df['BB_Upper'], label='Upper Band', color='red')
    ax.plot(df.index, df['BB_Lower'], label='Lower Band', color='green')
    ax.fill_between(df.index, df['BB_Lower'], df['BB_Upper'], alpha=0.1)
    ax.scatter(df.index[df['Buy_Signal_BB']], df['Close'][df['Buy_Signal_BB']],
              marker='^', color='green', label='Buy Signal', alpha=1)
    ax.scatter(df.index[df['Sell_Signal_BB']], df['Close'][df['Sell_Signal_BB']],
              marker='v', color='red', label='Sell Signal', alpha=1)
    ax.set_title('Bollinger Bands')
    ax.set_ylabel('Price')
    ax.legend()
    ax.grid(True, alpha=0.3)
    st.pyplot(fig)

    # MACD with Buy/Sell Signals
    st.subheader("MACD Indicator with Buy/Sell Signals")
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df.index, df['MACD'], label='MACD', color='blue')
    ax.plot(df.index, df['MACD_Signal'], label='Signal Line', color='red')
    ax.bar(df.index, df['MACD'] - df['MACD_Signal'], color=df.apply(
        lambda x: 'green' if x['MACD'] > x['MACD_Signal'] else 'red', axis=1),
        alpha=0.5, label='Histogram')
    ax.set_title('Moving Average Convergence Divergence (MACD)')
    ax.set_ylabel('MACD Value')
    ax.legend()
    ax.grid(True, alpha=0.3)
    st.pyplot(fig)

    # Moving Averages
    st.subheader("Moving Averages")
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df.index, df['Close'], color='k', label='Close Price')
    ax.plot(df.index, df['50_SMA'], color='b', label='50-day SMA')
    ax.plot(df.index, df['200_SMA'], color='g', label='200-day SMA')
    # Highlight golden cross and death cross
    golden_cross = (df['50_SMA'] > df['200_SMA']) & (df['50_SMA'].shift() <= df['200_SMA'].shift())
    death_cross = (df['50_SMA'] < df['200_SMA']) & (df['50_SMA'].shift() >= df['200_SMA'].shift())
    if golden_cross.any():
        golden_dates = df.index[golden_cross]
        for date in golden_dates:
            ax.axvline(x=date, color='gold', linestyle='--', alpha=0.7)
    if death_cross.any():
        death_dates = df.index[death_cross]
        for date in death_dates:
            ax.axvline(x=date, color='darkred', linestyle='--', alpha=0.7)
    ax.set_title('Moving Averages (50-day and 200-day)')
    ax.set_ylabel('Price')
    ax.legend()
    ax.grid(True, alpha=0.3)
    st.pyplot(fig)

    # Technical Analysis Summary
    st.subheader("Technical Analysis Summary")

    latest = df.iloc[-1]
    col1, col2, col3 = st.columns(3)

    with col1:
        rsi_status = "Overbought" if latest['RSI'] > 70 else "Oversold" if latest['RSI'] < 30 else "Neutral"
        st.metric("RSI Value", f"{latest['RSI']:.2f}", delta=rsi_status)

    with col2:
        bb_position = ((latest['Close'] - latest['BB_Lower']) / (latest['BB_Upper'] - latest['BB_Lower'])) * 100
        bb_status = "Upper Band" if bb_position > 80 else "Lower Band" if bb_position < 20 else "Middle"
        st.metric("Bollinger Position", f"{bb_position:.1f}%", delta=bb_status)

    with col3:
        macd_signal = "Bullish" if latest['MACD'] > latest['MACD_Signal'] else "Bearish"
        macd_diff = latest['MACD'] - latest['MACD_Signal']
        st.metric("MACD Signal", macd_signal, delta=f"{macd_diff:.4f}")

    # Technical signals summary
    signal_df = pd.DataFrame({
        'Indicator': ['RSI', 'Bollinger Bands', 'MACD', 'Moving Averages'],
        'Value': [
            f"{latest['RSI']:.2f}",
            f"Width: {((latest['BB_Upper'] - latest['BB_Lower'])/latest['Close']*100):.2f}%",
            f"MACD: {latest['MACD']:.4f}, Signal: {latest['MACD_Signal']:.4f}",
            f"50-day: {latest['50_SMA']:.2f}, 200-day: {latest['200_SMA']:.2f}"
        ],
        'Signal': [
            "Buy" if latest['Buy_Signal_RSI'] else "Sell" if latest['Sell_Signal_RSI'] else "Hold",
            "Buy" if latest['Buy_Signal_BB'] else "Sell" if latest['Sell_Signal_BB'] else "Hold",
            "Buy" if latest['Buy_Signal_MACD'] else "Sell",
            "Bullish" if latest['50_SMA'] > latest['200_SMA'] else "Bearish"
        ]
    })

    st.table(signal_df)

    # Data Preview
    st.subheader("Recent Data and Indicators")
    st.dataframe(df.tail())

# Sidebar inputs
st.sidebar.header('Settings')
selected_stock = st.sidebar.selectbox("Select Stock", list(stocks.keys()))
analysis_mode = st.sidebar.selectbox("Analysis Mode", ["Forecast", "Technical Analysis", "Both"])

# Technical analysis settings
if analysis_mode in ["Technical Analysis", "Both"]:
    st.sidebar.subheader("Technical Analysis Settings")
    ta_period = st.sidebar.selectbox(
        "Technical Analysis Period",
        ["1mo", "3mo", "6mo", "1y", "2y", "5y"],
        index=3
    )

# Forecast settings
if analysis_mode in ["Forecast", "Both"]:
    st.sidebar.subheader("Forecast Settings")
    n_years = st.sidebar.slider("Prediction Years", 1, 4, 2)
    period = n_years * 365

# Main app flow
try:
    # Fetch data
    ticker = stocks[selected_stock]

    if analysis_mode == "Forecast":
        df_train, raw_data = fetch_stock_data(ticker)
        show_forecast = True
        show_technical = False
    elif analysis_mode == "Technical Analysis":
        df_tech = fetch_stock_data(ticker, ta_period)
        df_tech = perform_technical_analysis(df_tech)
        show_forecast = False
        show_technical = True
    else:  # Both
        df_train, raw_data = fetch_stock_data(ticker)
        df_tech = fetch_stock_data(ticker, ta_period)
        df_tech = perform_technical_analysis(df_tech)
        show_forecast = True
        show_technical = True

    # Display Technical Analysis if selected
    if show_technical:
        display_technical_analysis(df_tech, selected_stock)

        if not show_forecast:
            # Disclaimer
            st.info("""
            📊 **Disclaimer**: This technical analysis is for educational purposes only. Stock markets are influenced by many factors
            not captured in these indicators. Always conduct thorough research and consult with financial advisors before making
            investment decisions.
            """)

    # Display Forecast if selected
    if show_forecast:
        st.header("Stock Price Forecast")

        # Display metrics
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("Current Price", f"${df_train['y'].iloc[-1]:.2f}")
        with col2:
            st.metric("Trading Days", len(df_train))
        with col3:
            price_change = ((df_train['y'].iloc[-1] - df_train['y'].iloc[0]) / df_train['y'].iloc[0]) * 100
            st.metric("Total Return", f"{price_change:.1f}%")

        # Historical price chart
        st.subheader("Historical Price Data")
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=df_train["ds"],
            y=df_train["y"],
            name="Price",
            line=dict(color='royalblue', width=1.5)
        ))

        if len(df_train) >= 30:
            ma30 = df_train['y'].rolling(window=30).mean()
            fig.add_trace(go.Scatter(
                x=df_train["ds"][29:],
                y=ma30[29:],
                name="30-Day MA",
                line=dict(color='orange', width=1.5, dash='dot')
            ))

        fig.update_layout(
            title=f"{selected_stock} Price History",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
            template="plotly_white",
            xaxis_rangeslider_visible=True
        )
        st.plotly_chart(fig, use_container_width=True)

        # Forecast
        st.subheader("Price Forecast")
        with st.spinner("Generating forecast..."):
            m = Prophet(
                yearly_seasonality=True,
                weekly_seasonality=True,
                daily_seasonality=False,
                changepoint_prior_scale=0.05
            )
            m.fit(df_train)
            future = m.make_future_dataframe(periods=period)
            forecast = m.predict(future)

        # Forecast results
        col1, col2 = st.columns(2)
        with col1:
            st.write("Latest Predictions")
            forecast_tail = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
            forecast_tail.columns = ['Date', 'Forecast', 'Lower', 'Upper']
            st.dataframe(forecast_tail)

        with col2:
            current_price = df_train['y'].iloc[-1]
            last_prediction = forecast['yhat'].iloc[-1]
            price_change = ((last_prediction - current_price) / current_price) * 100
            st.metric(
                "Predicted Change",
                f"{price_change:.1f}%",
                delta=f"${last_prediction - current_price:.2f}"
            )

        # Forecast plot
        fig1 = plot_plotly(m, forecast)
        fig1.update_layout(
            title=f"{selected_stock} Forecast - {n_years} Years",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
            template="plotly_white"
        )
        st.plotly_chart(fig1, use_container_width=True)

        # Components plot
        st.subheader("Forecast Components")
        fig2 = m.plot_components(forecast)
        st.write(fig2)

        # Disclaimer for forecast
        st.info("""
        📊 **Disclaimer**: This forecast is for educational purposes only. Stock markets are influenced by many factors
        not captured in this model. Always conduct thorough research and consult with financial advisors before making
        investment decisions.
        """)

except Exception as e:
    st.error(f"An error occurred: {str(e)}")
    st.code(f"Error details: {str(e)}")

# Footer
st.markdown("""
---
Created with ❤️ using Streamlit, Facebook Prophet, and Technical Indicators
""")

Writing stcokfinal.py


In [ ]:
!streamlit run stcokfinal.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴⠦
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.12.253:8501

⠧⠇⠏⠋⠙your url is: https://dry-dryers-ask.loca.lt
YF.download() has changed argument auto_adjust default to True
[*********************100%***********************]  1 of 1 completed
07:28:05 - cmdstanpy - INFO - Chain [1] start processing
07:28:05 - cmdstanpy - INFO - Chain [1] done processing
[*********************100%***********************]  1 of 1 completed
07:28:40 - cmdstanpy - INFO - Chain [1] start processing
07:28:40 - cmdstanpy - INFO - Chain [1] done processing
  Stopping...
^C


In [ ]:
%%writefile technicalanalysis.py
import streamlit as st
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import MACD

def trend(x):
    """Classify the trend based on percentage change"""
    if x > -0.5 and x <= 0.5:
        return 'Slight or No change'
    elif x > 0.5 and x <= 1:
        return 'Slight Positive'
    elif x > -1 and x <= -0.5:
        return 'Slight Negative'
    elif x > 1 and x <= 3:
        return 'Positive'
    elif x > -3 and x <= -1:
        return 'Negative'
    elif x > 3 and x <= 7:
        return 'Among top gainers'
    elif x > -7 and x <= -3:
        return 'Among top losers'
    elif x > 7:
        return 'Bull run'
    elif x <= -7:
        return 'Bear drop'

def rsi_signals(df):
    """Calculate RSI and corresponding buy/sell signals"""
    rsi = RSIIndicator(df['Close'], window=14)
    df['RSI'] = rsi.rsi()
    df['Buy_Signal_RSI'] = df['RSI'] < 40
    df['Sell_Signal_RSI'] = df['RSI'] > 70
    return df

def bollinger_signals(df):
    """Calculate Bollinger Bands and corresponding buy/sell signals"""
    bb = BollingerBands(df['Close'], window=20, window_dev=2)
    df['BB_Upper'] = bb.bollinger_hband()
    df['BB_Lower'] = bb.bollinger_lband()
    df['BB_Middle'] = bb.bollinger_mavg()
    df['Buy_Signal_BB'] = df['Close'] < df['BB_Lower']
    df['Sell_Signal_BB'] = df['Close'] > df['BB_Upper']
    return df

def macd_signals(df):
    """Calculate MACD and corresponding buy/sell signals"""
    macd = MACD(df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    df['MACD_Histogram'] = macd.macd_diff()
    df['Buy_Signal_MACD'] = df['MACD'] > df['MACD_Signal']
    df['Sell_Signal_MACD'] = df['MACD'] < df['MACD_Signal']
    return df

def calculate_moving_averages(df):
    """Calculate various moving averages"""
    df['50_SMA'] = df['Close'].rolling(window=50, min_periods=1).mean()
    df['200_SMA'] = df['Close'].rolling(window=200, min_periods=1).mean()
    df['20_EMA'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['50_EMA'] = df['Close'].ewm(span=50, adjust=False).mean()
    return df

def analyze(df, ticker_name):
    """Perform complete technical analysis and display results"""
    # Calculate all indicators
    df['Day_Perc_Change'] = df['Close'].pct_change() * 100
    df.dropna(inplace=True)
    df['Trend'] = df['Day_Perc_Change'].apply(lambda x: trend(x))
    df = rsi_signals(df)
    df = bollinger_signals(df)
    df = macd_signals(df)
    df = calculate_moving_averages(df)

    # Plot Market Trends
    st.subheader("Market Trend Analysis")
    col1, col2 = st.columns(2)
    with col1:
        fig, ax = plt.subplots(figsize=(8, 6))
        trend_counts = df['Trend'].value_counts()
        colors = plt.cm.Paired(np.linspace(0, 1, len(trend_counts)))
        trend_counts.plot.pie(autopct='%1.1f%%', ax=ax, colors=colors)
        plt.title('Distribution of Price Movements')
        st.pyplot(fig)

    with col2:
        st.markdown("### Trend Interpretation")
        st.write("""
        This pie chart shows the distribution of daily price movements categorized by trend strength.
        - **Bull run/Bear drop**: Extreme price movements (>7%)
        - **Top gainers/losers**: Strong movements (3-7%)
        - **Positive/Negative**: Moderate movements (1-3%)
        - **Slight changes**: Minor movements (0.5-1%)
        - **No change**: Minimal price movement (<0.5%)
        """)

    # RSI with Buy/Sell Signals
    st.subheader("RSI Indicator with Buy/Sell Signals")
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df.index, df['RSI'], label='RSI', color='blue')
    ax.axhline(70, linestyle='--', color='red', label='Overbought (70)')
    ax.axhline(30, linestyle='--', color='green', label='Oversold (30)')
    ax.scatter(df.index[df['Buy_Signal_RSI']], df['RSI'][df['Buy_Signal_RSI']],
              marker='^', color='green', label='Buy Signal', alpha=1)
    ax.scatter(df.index[df['Sell_Signal_RSI']], df['RSI'][df['Sell_Signal_RSI']],
              marker='v', color='red', label='Sell Signal', alpha=1)
    ax.set_title('Relative Strength Index (RSI)')
    ax.set_ylabel('RSI Value')
    ax.legend()
    ax.grid(True, alpha=0.3)
    st.pyplot(fig)

    # Bollinger Bands with Buy/Sell Signals
    st.subheader("Bollinger Bands with Buy/Sell Signals")
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df.index, df['Close'], label='Close Price', color='black')
    ax.plot(df.index, df['BB_Upper'], label='Upper Band', color='red')
    ax.plot(df.index, df['BB_Middle'], label='Middle Band', color='blue', linestyle='--')
    ax.plot(df.index, df['BB_Lower'], label='Lower Band', color='green')
    ax.fill_between(df.index, df['BB_Lower'], df['BB_Upper'], alpha=0.1)
    ax.scatter(df.index[df['Buy_Signal_BB']], df['Close'][df['Buy_Signal_BB']],
              marker='^', color='green', label='Buy Signal', alpha=1)
    ax.scatter(df.index[df['Sell_Signal_BB']], df['Close'][df['Sell_Signal_BB']],
              marker='v', color='red', label='Sell Signal', alpha=1)
    ax.set_title('Bollinger Bands')
    ax.set_ylabel('Price')
    ax.legend()
    ax.grid(True, alpha=0.3)
    st.pyplot(fig)

    # MACD with Buy/Sell Signals
    st.subheader("MACD Indicator with Buy/Sell Signals")
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), gridspec_kw={'height_ratios': [2, 1]})

    # MACD and Signal line
    ax1.plot(df.index, df['MACD'], label='MACD', color='blue')
    ax1.plot(df.index, df['MACD_Signal'], label='Signal Line', color='red')
    ax1.scatter(df.index[df['Buy_Signal_MACD']], df['MACD'][df['Buy_Signal_MACD']],
               marker='^', color='green', label='Buy Signal', alpha=1)
    ax1.scatter(df.index[df['Sell_Signal_MACD']], df['MACD'][df['Sell_Signal_MACD']],
               marker='v', color='red', label='Sell Signal', alpha=1)
    ax1.set_title('MACD and Signal Line')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # MACD Histogram
    colors = ['green' if val >= 0 else 'red' for val in df['MACD_Histogram']]
    ax2.bar(df.index, df['MACD_Histogram'], color=colors, alpha=0.7)
    ax2.set_title('MACD Histogram')
    ax2.grid(True, alpha=0.3)

    fig.tight_layout()
    st.pyplot(fig)

    # Moving Averages
    st.subheader("Moving Averages")
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df.index, df['Close'], color='k', label='Close Price')
    ax.plot(df.index, df['50_SMA'], color='b', label='50-day SMA')
    ax.plot(df.index, df['200_SMA'], color='g', label='200-day SMA')
    ax.plot(df.index, df['20_EMA'], color='orange', linestyle='--', label='20-day EMA')

    # Highlight golden cross and death cross
    golden_cross = (df['50_SMA'] > df['200_SMA']) & (df['50_SMA'].shift() <= df['200_SMA'].shift())
    death_cross = (df['50_SMA'] < df['200_SMA']) & (df['50_SMA'].shift() >= df['200_SMA'].shift())

    if golden_cross.any():
        golden_dates = df.index[golden_cross]
        for date in golden_dates:
            ax.axvline(x=date, color='gold', linestyle='--', alpha=0.7, label='Golden Cross' if date == golden_dates[0] else "")

    if death_cross.any():
        death_dates = df.index[death_cross]
        for date in death_dates:
            ax.axvline(x=date, color='darkred', linestyle='--', alpha=0.7, label='Death Cross' if date == death_dates[0] else "")

    ax.set_title('Moving Averages (50-day and 200-day)')
    ax.set_ylabel('Price')
    ax.legend()
    ax.grid(True, alpha=0.3)
    st.pyplot(fig)

    # Technical Analysis Summary
    st.subheader("Technical Analysis Summary")

    latest = df.iloc[-1]
    col1, col2, col3 = st.columns(3)

    with col1:
        rsi_status = "Overbought" if latest['RSI'] > 70 else "Oversold" if latest['RSI'] < 30 else "Neutral"
        st.metric("RSI Value", f"{latest['RSI']:.2f}", delta=rsi_status)

    with col2:
        bb_position = ((latest['Close'] - latest['BB_Lower']) / (latest['BB_Upper'] - latest['BB_Lower'])) * 100
        bb_status = "Upper Band" if bb_position > 80 else "Lower Band" if bb_position < 20 else "Middle"
        st.metric("Bollinger Position", f"{bb_position:.1f}%", delta=bb_status)

    with col3:
        macd_signal = "Bullish" if latest['MACD'] > latest['MACD_Signal'] else "Bearish"
        macd_diff = latest['MACD'] - latest['MACD_Signal']
        st.metric("MACD Signal", macd_signal, delta=f"{macd_diff:.4f}")

    # Technical signals summary
    st.subheader("Trading Signals Summary")

    # Create a nice-looking table for signals
    signal_df = pd.DataFrame({
        'Indicator': ['RSI', 'Bollinger Bands', 'MACD', 'Moving Averages'],
        'Current Value': [
            f"{latest['RSI']:.2f}",
            f"Width: {((latest['BB_Upper'] - latest['BB_Lower'])/latest['Close']*100):.2f}%",
            f"MACD: {latest['MACD']:.4f}, Signal: {latest['MACD_Signal']:.4f}",
            f"50-day SMA: {latest['50_SMA']:.2f}, 200-day SMA: {latest['200_SMA']:.2f}"
        ],
        'Signal': [
            "Buy" if latest['Buy_Signal_RSI'] else "Sell" if latest['Sell_Signal_RSI'] else "Hold",
            "Buy" if latest['Buy_Signal_BB'] else "Sell" if latest['Sell_Signal_BB'] else "Hold",
            "Buy" if latest['Buy_Signal_MACD'] else "Sell",
            "Bullish" if latest['50_SMA'] > latest['200_SMA'] else "Bearish"
        ],
        'Interpretation': [
            "Oversold (<40)" if latest['RSI'] < 40 else "Overbought (>70)" if latest['RSI'] > 70 else "Neutral zone",
            "Price below lower band" if latest['Buy_Signal_BB'] else "Price above upper band" if latest['Sell_Signal_BB'] else "Price within bands",
            "MACD above Signal Line" if latest['Buy_Signal_MACD'] else "MACD below Signal Line",
            "Golden Cross" if latest['50_SMA'] > latest['200_SMA'] and latest['50_SMA'].shift() <= latest['200_SMA'].shift() else
            "Death Cross" if latest['50_SMA'] < latest['200_SMA'] and latest['50_SMA'].shift() >= latest['200_SMA'].shift() else
            "Uptrend" if latest['50_SMA'] > latest['200_SMA'] else "Downtrend"
        ]
    })

    # Display styled dataframe
    st.dataframe(signal_df.set_index('Indicator'))

    # Trend strength visualization
    st.subheader("Recent Trend Strength")
    recent_df = df.tail(30).copy()

    # Calculate trend strength using RSI, MACD, and price position relative to MAs
    recent_df['Trend_Score'] = 0

    # RSI contribution: Higher RSI means stronger uptrend
    recent_df['Trend_Score'] += (recent_df['RSI'] - 50) / 50 * 3

    # MACD contribution: Positive MACD histogram means uptrend
    recent_df['Trend_Score'] += recent_df['MACD_Histogram'] / recent_df['Close'].mean() * 100

    # MA contribution: Price above MAs means uptrend
    recent_df['Trend_Score'] += ((recent_df['Close'] - recent_df['50_SMA']) / recent_df['50_SMA']) * 5
    recent_df['Trend_Score'] += ((recent_df['Close'] - recent_df['200_SMA']) / recent_df['200_SMA']) * 5

    # Plot trend strength
    fig, ax = plt.subplots(figsize=(12, 6))
    colors = ['green' if x >= 0 else 'red' for x in recent_df['Trend_Score']]
    ax.bar(recent_df.index, recent_df['Trend_Score'], color=colors, alpha=0.7)
    ax.axhline(0, color='black', linestyle='-', alpha=0.3)
    ax.set_title('Trend Strength Indicator (Last 30 Days)')
    ax.set_ylabel('Strength Score (+ is bullish, - is bearish)')
    plt.xticks(rotation=45)
    st.pyplot(fig)

    # Data Preview
    st.subheader("Recent Data and Indicators")
    st.dataframe(df.tail())

    # Return the dataframe with all indicators for further use if needed
    return df

Writing technicalanalysis.py


In [ ]:
%%writefile app.py
import streamlit as st
from datetime import date, datetime, timedelta
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.plot import plot_plotly
from plotly import graph_objs as go
import matplotlib.pyplot as plt
import technicalanalysis as ta

# Configure page
st.set_page_config(
    page_title="Stock Analysis & Forecast App",
    page_icon="📈",
    layout="wide"
)

# App title and description
st.title('📈 Stock Analysis & Forecast App')
st.markdown("""
This app provides comprehensive stock analysis using technical indicators and forecasts future prices using Facebook's Prophet model.
""")

# Define available stock options
stocks = {
    "Google": "GOOGL",
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Tesla": "TSLA",
    "Tata Motors": "TATAMOTORS.NS"
}

def fetch_data_from_yfinance(ticker, period='5y'):
    """Fetch stock data using yfinance"""
    try:
        # Get data for the specified period
        end_date = datetime.now()

        if period == '5y' or period == '1825d':
            start_date = end_date - timedelta(days=1825)
        else:
            # For technical analysis periods
            df = yf.download(ticker, period=period)
            return df

        # Download data for forecast
        df = yf.download(ticker, start=start_date, end=end_date)

        if not df.empty and len(df) > 30:
            return df
        return None
    except Exception as e:
        st.error(f"Error fetching data from Yahoo Finance: {str(e)}")
        return None

def get_sample_data(ticker):
    """Generate sample stock data for demonstration"""
    seed_value = sum(ord(c) for c in ticker)
    np.random.seed(seed_value)

    end_date = datetime.now()
    start_date = end_date - timedelta(days=1825)
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')

    # Set initial price based on ticker
    start_price = {
        'AAPL': 150,
        'GOOGL': 2800,
        'MSFT': 300,
        'TSLA': 800
    }.get(ticker, 100)

    # Generate price data
    daily_returns = np.random.normal(0.0005, 0.02, len(date_range))
    price_series = start_price * (1 + daily_returns).cumprod()

    # Create DataFrame
    df = pd.DataFrame(index=date_range)
    df['Close'] = price_series
    df['Open'] = df['Close'].shift(1) * (1 + np.random.normal(0, 0.005, len(df)))
    df['High'] = df[['Open', 'Close']].max(axis=1) * (1 + abs(np.random.normal(0, 0.003, len(df))))
    df['Low'] = df[['Open', 'Close']].min(axis=1) * (1 - abs(np.random.normal(0, 0.003, len(df))))
    df['Adj Close'] = df['Close']
    df['Volume'] = np.random.randint(100000, 10000000, len(df))

    return df.fillna(method='bfill')

def prepare_dataframe(df):
    """Prepare dataframe for Prophet model"""
    df = df.reset_index()
    result_df = pd.DataFrame()
    result_df['ds'] = df['Date']
    result_df['y'] = df['Close']
    return result_df

@st.cache_data(show_spinner=False)
def fetch_stock_data(stock_ticker, period='5y'):
    """Fetch stock data with fallback to sample data"""
    with st.spinner(f"Fetching data for {stock_ticker}..."):
        df = fetch_data_from_yfinance(stock_ticker, period)
        if df is not None and not df.empty and len(df) > 30:
            st.success(f"Successfully loaded data for {stock_ticker}")
            if period == '5y' or period == '1825d':
                return prepare_dataframe(df), df
            return df

        st.warning(f"Using sample data for {stock_ticker}")
        sample_data = get_sample_data(stock_ticker)
        if period == '5y' or period == '1825d':
            return prepare_dataframe(sample_data), sample_data
        return sample_data

def display_candlestick_chart(df, title):
    """Display a candlestick chart for the given stock data"""
    fig = go.Figure()

    # Add candlestick chart
    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name="Candlestick"
    ))

    # Add volume as a bar chart at the bottom
    fig.add_trace(go.Bar(
        x=df.index,
        y=df['Volume'],
        name="Volume",
        marker_color='rgba(0, 0, 255, 0.3)',
        opacity=0.3,
        yaxis="y2"
    ))

    # Add a 30-day moving average
    if len(df) >= 30:
        ma30 = df['Close'].rolling(window=30).mean()
        fig.add_trace(go.Scatter(
            x=df.index,
            y=ma30,
            name="30-Day MA",
            line=dict(color='orange', width=1.5)
        ))

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis_title="Price",
        yaxis2=dict(
            title="Volume",
            titlefont=dict(color="blue"),
            tickfont=dict(color="blue"),
            anchor="x",
            overlaying="y",
            side="right",
            showgrid=False
        ),
        template="plotly_white",
        xaxis_rangeslider_visible=True,
        height=600
    )

    return fig

# Sidebar inputs
st.sidebar.header('Settings')
selected_stock = st.sidebar.selectbox("Select Stock", list(stocks.keys()))
analysis_mode = st.sidebar.selectbox("Analysis Mode", ["Forecast", "Technical Analysis", "Both"])

# Technical analysis settings
if analysis_mode in ["Technical Analysis", "Both"]:
    st.sidebar.subheader("Technical Analysis Settings")
    ta_period = st.sidebar.selectbox(
        "Technical Analysis Period",
        ["1mo", "3mo", "6mo", "1y", "2y", "5y"],
        index=3
    )

# Forecast settings
if analysis_mode in ["Forecast", "Both"]:
    st.sidebar.subheader("Forecast Settings")
    n_years = st.sidebar.slider("Prediction Years", 1, 4, 2)
    period = n_years * 365

# Main app flow
try:
    # Fetch data
    ticker = stocks[selected_stock]

    if analysis_mode == "Forecast":
        df_train, raw_data = fetch_stock_data(ticker)
        show_forecast = True
        show_technical = False
    elif analysis_mode == "Technical Analysis":
        df_tech = fetch_stock_data(ticker, ta_period)
        show_forecast = False
        show_technical = True
    else:  # Both
        df_train, raw_data = fetch_stock_data(ticker)
        df_tech = fetch_stock_data(ticker, ta_period)
        show_forecast = True
        show_technical = True

    # Display Technical Analysis if selected
    if show_technical:
        st.header(f"Technical Analysis for {selected_stock}")

        # Use the imported technicalanalysis module
        with st.spinner("Calculating technical indicators..."):
            # Display candlestick chart first
            st.subheader("Price History (Candlestick Chart)")
            candlestick_fig = display_candlestick_chart(df_tech, f"{selected_stock} Price History")
            st.plotly_chart(candlestick_fig, use_container_width=True)

            # Run technical analysis using the imported module
            ta.analyze(df_tech, ticker)

        if not show_forecast:
            # Disclaimer
            st.info("""
            📊 **Disclaimer**: This technical analysis is for educational purposes only. Stock markets are influenced by many factors
            not captured in these indicators. Always conduct thorough research and consult with financial advisors before making
            investment decisions.
            """)

    # Display Forecast if selected
    if show_forecast:
        st.header("Stock Price Forecast")

        # Display metrics
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("Current Price", f"${df_train['y'].iloc[-1]:.2f}")
        with col2:
            st.metric("Trading Days", len(df_train))
        with col3:
            price_change = ((df_train['y'].iloc[-1] - df_train['y'].iloc[0]) / df_train['y'].iloc[0]) * 100
            st.metric("Total Return", f"{price_change:.1f}%")

        # Historical price chart as candlestick
        st.subheader("Historical Price Data")
        candlestick_fig = display_candlestick_chart(raw_data, f"{selected_stock} Price History")
        st.plotly_chart(candlestick_fig, use_container_width=True)

        # Forecast
        st.subheader("Price Forecast")
        with st.spinner("Generating forecast..."):
            m = Prophet(
                yearly_seasonality=True,
                weekly_seasonality=True,
                daily_seasonality=False,
                changepoint_prior_scale=0.05
            )
            m.fit(df_train)
            future = m.make_future_dataframe(periods=period)
            forecast = m.predict(future)

        # Forecast results
        col1, col2 = st.columns(2)
        with col1:
            st.write("Latest Predictions")
            forecast_tail = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
            forecast_tail.columns = ['Date', 'Forecast', 'Lower', 'Upper']
            st.dataframe(forecast_tail)

        with col2:
            current_price = df_train['y'].iloc[-1]
            last_prediction = forecast['yhat'].iloc[-1]
            price_change = ((last_prediction - current_price) / current_price) * 100
            st.metric(
                "Predicted Change",
                f"{price_change:.1f}%",
                delta=f"${last_prediction - current_price:.2f}"
            )

        # Forecast plot
        fig1 = plot_plotly(m, forecast)
        fig1.update_layout(
            title=f"{selected_stock} Forecast - {n_years} Years",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
            template="plotly_white"
        )
        st.plotly_chart(fig1, use_container_width=True)

        # Components plot
        st.subheader("Forecast Components")
        fig2 = m.plot_components(forecast)
        st.write(fig2)

        # Disclaimer for forecast
        st.info("""
        📊 **Disclaimer**: This forecast is for educational purposes only. Stock markets are influenced by many factors
        not captured in this model. Always conduct thorough research and consult with financial advisors before making
        investment decisions.
        """)

except Exception as e:
    st.error(f"An error occurred: {str(e)}")
    st.code(f"Error details: {str(e)}")

# Footer
st.markdown("""
---
Created with ❤️ using Streamlit, Facebook Prophet, and Technical Indicators
""")

Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.12.253:8501

⠴your url is: https://fifty-coats-hang.loca.lt
2025-03-02 07:40:05.990 Uncaught app execution
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/runtime/scriptrunner/exec_code.py", line 121, in exec_func_with_error_handling
    result = func()
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 591, in code_to_exec
    exec(code, module.__dict__)
  File "/content/app.py", line 13, in <module>
    st.set_page_config(
  File "/usr/local/lib/python3.11/dist-packages/streamlit/runtime/metrics_util.py", line 410, in wrapped_func
    result = non_optional_func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/streamlit/commands/page_config.py

In [ ]:
##############
#### Deep Charts Youtube Channel: https://www.youtube.com/@DeepCharts
#### Subscribe for more AI/Machine Learning/Quant Finance Tutorials
##############


##############
### PART 1 ###
# LIBRARIES ##
# & OLLAMA  ##
##############

%%writefile ollama.py

# Data Importing Libraries
import yfinance as yf
from finvizfinance.quote import finvizfinance

# Data Modeling Library
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Charts
import plotly.graph_objects as go

# Data Manipulation
import pandas as pd
import numpy as np

# Avoid Forecasting on Holidays
import holidays

# Create Local LLM Server Connection
from langchain_community.llms import Ollama

# Interactive Web App UI
import streamlit as st


# Connect to local Ollama server
llm = Ollama(model='llama3')


##############
### PART 2 ###
# FUNCTIONS ##
##############

# Function to classify sentiment
def classify_sentiment(title):
    output = llm.invoke(f"Classify the sentiment as 'POSITIVE' or 'NEGATIVE' or 'NEUTRAL' with just that one word only, no additional words or reasoning: {title}")
    return output.strip()  # Ensure the response is clean and without extra spaces

# Function to get and process news data
def get_news_data(ticker):

    # Data Pull
    stock = finvizfinance(ticker)
    news_df = stock.ticker_news()

    # Preprocess before putting into LLM
    news_df['Title'] = news_df['Title'].str.lower()

    # Classify Sentiment function applied to each row of news_df
    news_df['sentiment'] = news_df['Title'].apply(classify_sentiment)

    # Postprocess after putting into LLM
    news_df['sentiment'] = news_df['sentiment'].str.upper()
    news_df = news_df[news_df['sentiment'] != 'NEUTRAL']
    news_df['Date'] = pd.to_datetime(news_df['Date'])
    news_df['DateOnly'] = news_df['Date'].dt.date

    return news_df

# Function to group and process sentiment data
def process_sentiment_data(news_df):

    # Reshape data to have df with columns: Date, # of positive Articles, # of negative Articles
    grouped = news_df.groupby(['DateOnly', 'sentiment']).size().unstack(fill_value=0)
    grouped = grouped.reindex(columns=['POSITIVE', 'NEGATIVE'], fill_value=0)

    # Create rolling averages that count number of positive and negative sentiment articles within past t days
    grouped['7day_avg_positive'] = grouped['POSITIVE'].rolling(window=7, min_periods=1).sum()
    grouped['7day_avg_negative'] = grouped['NEGATIVE'].rolling(window=7, min_periods=1).sum()

    # Create "Percent Positive" by creating percentage measure
    grouped['7day_pct_positive'] = grouped['POSITIVE'] / (grouped['POSITIVE'] + grouped['NEGATIVE'])
    result_df = grouped.reset_index()

    return result_df

# Function to fetch and process stock data
def get_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date) # Pull ticker data
    stock_data['Pct_Change'] = stock_data['Close'].pct_change() * 100 # Transform closing value to percent change in closing value since previous day
    return stock_data

# Function to combine sentiment and stock data
def combine_data(result_df, stock_data):
    combined_df = result_df.set_index('DateOnly').join(stock_data[['Pct_Change']], how='inner')
    combined_df['lagged_7day_pct_positive'] = combined_df['7day_pct_positive'].shift(1) # Lag sentiment feature by 1 day for temporal alignment
    return combined_df

# Function to calculate Pearson correlation
def calculate_correlation(combined_df):
    correlation_pct_change = combined_df[['lagged_7day_pct_positive', 'Pct_Change']].corr().iloc[0, 1]
    return correlation_pct_change

# Function to get future dates excluding weekends and holidays
def get_future_dates(start_date, num_days):
    us_holidays = holidays.US()
    future_dates = []
    current_date = start_date
    while len(future_dates) < num_days:
        if current_date.weekday() < 5 and current_date not in us_holidays:
            future_dates.append(current_date)
        current_date += pd.Timedelta(days=1)
    return future_dates

# Function to fit ARIMAX model and forecast
def fit_and_forecast(combined_df, forecast_steps=3):
    endog = combined_df['Pct_Change'].dropna()  # Dependent variable
    exog = combined_df['lagged_7day_pct_positive'].dropna()  # Predictor variable
    endog = endog.loc[exog.index]  # Align variables
    model = SARIMAX(endog, exog=exog, order=(1, 1, 1))  # ARIMAX model
    fit = model.fit(disp=False)  # Fit model

    future_dates = get_future_dates(combined_df.index[-1], forecast_steps)  # Future dates
    future_exog = combined_df['lagged_7day_pct_positive'][-forecast_steps:].values.reshape(-1, 1)  # Future exogenous values

    forecast = fit.get_forecast(steps=forecast_steps, exog=future_exog)  # Get forecast
    forecast_mean = forecast.predicted_mean  # Predicted mean
    forecast_ci = forecast.conf_int()  # Confidence intervals

    return forecast_mean, forecast_ci, future_dates  # Return results


# Function to create and display plot
def create_plot(combined_df, forecast_mean, forecast_ci, forecast_index):
    # Standardize the sentiment proportion
    sentiment_std = (combined_df['7day_pct_positive'] - combined_df['7day_pct_positive'].mean()) / combined_df['7day_pct_positive'].std()

    fig = go.Figure()

    # Add standardized sentiment proportion
    fig.add_trace(go.Scatter(
        x=combined_df.index,
        y=sentiment_std,
        name='Standardized Sentiment Proportion',
        line=dict(color='blue'),
        mode='lines'
    ))

    # Add stock percentage change
    fig.add_trace(go.Scatter(
        x=combined_df.index,
        y=combined_df['Pct_Change'],
        name='Stock Pct Change',
        line=dict(color='green'),
        yaxis='y2',
        mode='lines'
    ))

    # Add forecasted stock percentage change
    fig.add_trace(go.Scatter(
        x=forecast_index,
        y=forecast_mean,
        name='Forecasted Pct Change',
        line=dict(color='red'),
        mode='lines'
    ))

    # Add confidence intervals for the forecast
    fig.add_trace(go.Scatter(
        x=np.concatenate([forecast_index, forecast_index[::-1]]),
        y=np.concatenate([forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1][::-1]]),
        fill='toself',
        fillcolor='rgba(255,0,0,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False
    ))

    # Update layout with appropriate y-axis ranges
    fig.update_layout(
        title='Sentiment Proportion and Stock Percentage Change with Forecast',
        xaxis_title='Date',
        yaxis=dict(
            title='Standardized Sentiment Proportion',
            titlefont=dict(color='blue')
        ),
        yaxis2=dict(
            title='Stock Pct Change',
            titlefont=dict(color='green'),
            overlaying='y',
            side='right'
        ),
        template='plotly_dark'
    )
    st.plotly_chart(fig)


##############
### PART 3 ###
# STREAMLIT ##
##############

# Streamlit app
st.sidebar.title("Predicting Stock Prices by News Sentiment")
ticker = st.sidebar.text_input("Enter stock ticker (e.g., SBUX):", value='SBUX')
run_button = st.sidebar.button("Run Analysis")

if run_button:
    news_df = get_news_data(ticker)
    result_df = process_sentiment_data(news_df)
    start_date = result_df['DateOnly'].min().strftime('%Y-%m-%d')
    end_date = result_df['DateOnly'].max().strftime('%Y-%m-%d')
    stock_data = get_stock_data(ticker, start_date, end_date)
    combined_df = combine_data(result_df, stock_data)
    correlation_pct_change = calculate_correlation(combined_df)
    st.write(f'Pearson correlation between lagged sentiment score and stock percentage change: {correlation_pct_change}')
    forecast_mean, forecast_ci, forecast_index = fit_and_forecast(combined_df)
    create_plot(combined_df, forecast_mean, forecast_ci, forecast_index)








Overwriting ollama.py


In [ ]:
%%writefile ollama.py
st.sidebar.title("Predicting Stock Prices by News Sentiment")
ticker = st.sidebar.text_input("Enter stock ticker (e.g., SBUX):", value='SBUX')
run_button = st.sidebar.button("Run Analysis")

if run_button:
    news_df = get_news_data(ticker)
    result_df = process_sentiment_data(news_df)
    start_date = result_df['DateOnly'].min().strftime('%Y-%m-%d')
    end_date = result_df['DateOnly'].max().strftime('%Y-%m-%d')
    stock_data = get_stock_data(ticker, start_date, end_date)
    combined_df = combine_data(result_df, stock_data)
    correlation_pct_change = calculate_correlation(combined_df)
    st.write(f'Pearson correlation between lagged sentiment score and stock percentage change: {correlation_pct_change}')
    forecast_mean, forecast_ci, forecast_index = fit_and_forecast(combined_df)
    create_plot(combined_df, forecast_mean, forecast_ci, forecast_index)


Writing ollama.py


In [ ]:
!pip install finvizfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00


In [ ]:
!pip install yfinance finvizfinance statsmodels plotly pandas numpy holidays langchain-community streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


In [ ]:
!streamlit run ollama.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.12.253:8501

⠴⠦⠧⠇⠏⠋⠙your url is: https://large-tigers-see.loca.lt
/content/ollama.py:27: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model='llama3')
2025-03-02 08:44:51.635 Uncaught app execution
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    

In [ ]:
!pip install streamlit